# Размер случайного леса

### Вы научитесь:

- работать со случайным лесом — одним из наиболее распространенных семейств алгоритмов
- решать с его помощью задачи регрессии
- подбирать параметры случайного леса

### Введение

Случайный лес — это модель классификации, объединяющая некоторое количество решающих деревьев в одну композицию, за счет чего улучшается их качество работы и обобщающая способность. Деревья строятся независимо друг от друга. Чтобы они отличались друг от друга, обучение проводится не на всей обучающей выборке, а на ее случайном подмножестве. Также, для дальнейшего уменьшения схожести деревьев, оптимальный признак для разбиения выбирается не из всех возможных признаков, а лишь из их случайного подмножества. Прогнозы, выданные деревьями, объединяются в один ответ путем усреднения.


Особенность случайного леса заключается в том, что он не переобучается по мере увеличения количества деревьев в композиции. Это достигается за счет того, что деревья не зависят друг от друга, и поэтому добавление нового дерева в композицию не усложняет модель, а лишь понижает уровень шума в прогнозах.

### Реализация в Scikit-Learn

В библиотеке `scikit-learn` случайные леса реализованы в классах `sklearn.ensemble.RandomForestClassifier` (для классификации) и `sklearn.ensemble.RandomForestRegressor` (для регрессии). Обучение модели производится с помощью функции `fit`, построение прогнозов — с помощью функции `predict`. Число деревьев задается с помощью поля класса `n_estimators`.

Пример использования:

```python
import numpy as np
from sklearn.ensemble import RandomForestRegressor
X = np.array([[1, 2], [3, 4], [5, 6]])
y = np.array([-3, 1, 10])
clf = RandomForestRegressor(n_estimators=100)
clf.fit(X, y)
predictions = clf.predict(X)
```

Также в этом задании вам понадобится вычислять качество предсказаний на тестовой выборке. Мы будем пользоваться метрикой R2 — по сути, это среднеквадратичная ошибка (MSE), нормированная на отрезок [0, 1] и обращенная так, чтобы ее наилучшим значением была единица. Ее можно вычислить с помощью функции `sklearn.metrics.r2_score`. Первым аргументов является список правильных ответов на выборке, вторым — список предсказанных ответов. Пример использования:

```python
from sklearn.metrics import r2_score
print r2_score([10, 11, 12], [9, 11, 12.1])
```

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

# 1
Загрузите данные из файла `abalone.csv`. Это датасет, в котором требуется предсказать возраст ракушки (число колец) по физическим измерениям.

In [2]:
data = pd.read_csv('abalone.csv')

In [3]:
data.head()

,Sex,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 9 columns):
Sex              4177 non-null object
Length           4177 non-null float64
Diameter         4177 non-null float64
Height           4177 non-null float64
WholeWeight      4177 non-null float64
ShuckedWeight    4177 non-null float64
VisceraWeight    4177 non-null float64
ShellWeight      4177 non-null float64
Rings            4177 non-null int64
dtypes: float64(7), int64(1), object(1)
memory usage: 293.8+ KB


# 2
Преобразуйте признак `Sex` в числовой: значение `F` должно перейти в `-1`, `I` — в `0`, `M` — в `1`. Если вы используете `Pandas`, то подойдет следующий код: `data['Sex'] = data['Sex'].map(lambda x: 1 if x == 'M' else (-1 if x == 'F' else 0))`

In [7]:
data['Sex'] = data['Sex'].map(lambda x: 1 if x == 'M' else (-1 if x == 'F' else 0))

In [8]:
data.head()

,Sex,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings
0,1,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,-1,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,1,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,0,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


# 3
Разделите содержимое файлов на признаки и целевую переменную. В последнем столбце записана целевая переменная, в остальных — признаки.

In [16]:
y = data.iloc[:, -1]
X = data.iloc[:, :-1]

In [17]:
y.head()

0    15
1     7
2     9
3    10
4     7
Name: Rings, dtype: int64

In [18]:
X.head()

,Sex,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight
0,1,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150
1,1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070
2,-1,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210
3,1,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155
4,0,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055


# 4
Обучите случайный лес (`sklearn.ensemble.RandomForestRegressor`) с различным числом деревьев: от 1 до 50 (не забудьте выставить "`random_state=1`" в конструкторе). Для каждого из вариантов оцените качество работы полученного леса на кросс-валидации по 5 блокам. Используйте параметры "`random_state=1`" и "`shuffle=True`" при создании генератора кросс-валидации `sklearn.cross_validation.KFold`. В качестве меры качества воспользуйтесь коэффициентом детерминации (`sklearn.metrics.r2_score`).

# 5
Определите, при каком минимальном количестве деревьев случайный лес показывает качество на кросс-валидации выше 0.52. Это количество и будет ответом на задание.

In [23]:
kf = KFold(n_splits=5, random_state=1, shuffle=True)
    
for i in range(1, 51):
    regr = RandomForestRegressor(n_estimators=i, random_state=1)
    scores = np.array([])
    
    for train_index, test_index in kf.split(X):
        regr.fit(X.loc[train_index], y.loc[train_index])
        prediction = regr.predict(X.loc[test_index])
        scores = np.append(scores, r2_score(y.loc[test_index], prediction))

    current_score = scores.mean()
    print('n_estimators: ', i, ' Score: ', current_score)
    
    if current_score > 0.52:
        f = open('submission.txt', 'w')
        f.write(str(i))
        f.close()
        break

n_estimators:  1  Score:  0.102138694877
n_estimators:  2  Score:  0.338416755158
n_estimators:  3  Score:  0.403579849462
n_estimators:  4  Score:  0.442722398967
n_estimators:  5  Score:  0.464020766067
n_estimators:  6  Score:  0.470581632759
n_estimators:  7  Score:  0.475830616352
n_estimators:  8  Score:  0.481741845626
n_estimators:  9  Score:  0.488347813022
n_estimators:  10  Score:  0.494464124802
n_estimators:  11  Score:  0.493396555
n_estimators:  12  Score:  0.497965876398
n_estimators:  13  Score:  0.502136460572
n_estimators:  14  Score:  0.506428696226
n_estimators:  15  Score:  0.508331197043
n_estimators:  16  Score:  0.510513143832
n_estimators:  17  Score:  0.513848294799
n_estimators:  18  Score:  0.516327541274
n_estimators:  19  Score:  0.519034688136
n_estimators:  20  Score:  0.518673592872
n_estimators:  21  Score:  0.519835423354
n_estimators:  22  Score:  0.520158353672
